In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
import re
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score, precision_score, f1_score

In [2]:
vids = pd.read_csv('/content/drive/MyDrive/Data/Copy of USvideos.csv')
dict1 = json.load(open('/content/drive/MyDrive/Data/Copy of US_category_id.json'))
cats = []
for i in dict1['items']:
  cats.append([i['id'], i['snippet']['title']])
cats = pd.DataFrame(cats, columns = ['ID', 'Title'])
vids = vids.astype({'category_id': 'int32'}, copy = False)
cats = cats.astype({'ID': 'int32'}, copy = False)
vids = vids[['title', 'category_id']]
data = vids[['title', 'category_id']].merge(cats, left_on = 'category_id', right_on = 'ID')
data_x = data['title']
data_y = data['Title']
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size = 0.2)

In [3]:
glove = {}
with open('/content/drive/MyDrive/Data/glove.6B.100d','r') as f:
  for line in f:
    split_line = line.split()
    word = split_line[0]
    embedding = np.array(split_line[1:], dtype=np.float64)
    glove[word] = embedding
  print(f"{len(glove)} words loaded!")

400000 words loaded!


In [18]:
inputs_train = pd.DataFrame([[]])
for i in range(len(train_x)):
  vecs = np.zeros(100)
  count = 0
  for word in train_x.iloc[i]:
    word = word.lower()
    word = re.sub(r'[^\w\s]', '', word)
    try: 
      vecs += glove[word]
      count += 1
    except KeyError:
      pass
  vecs /= count
  vecs = pd.DataFrame([vecs])
  inputs_train = pd.concat([inputs_train, vecs])
inputs_train = inputs_train.tail(-1)
inputs_train.replace(np.nan, 0, inplace = True)

inputs_test = pd.DataFrame([[]])
for i in range(len(test_x)):
  vecs = np.zeros(100)
  count = 0
  for word in test_x.iloc[i]:
    word = word.lower()
    word = re.sub(r'[^\w\s]', '', word)
    try: 
      vecs += glove[word]
      count += 1
    except KeyError:
      pass
  vecs /= count
  vecs = pd.DataFrame([vecs])
  inputs_test = pd.concat([inputs_test, vecs])
inputs_test = inputs_test.tail(-1)
inputs_test.replace(np.nan, 0, inplace = True)

<ipython-input-18-9457d1f36c6a>:13: RuntimeWarning: invalid value encountered in true_divide
  vecs /= count
<ipython-input-18-9457d1f36c6a>:13: RuntimeWarning: invalid value encountered in true_divide
  vecs /= count
<ipython-input-18-9457d1f36c6a>:13: RuntimeWarning: invalid value encountered in true_divide
  vecs /= count
<ipython-input-18-9457d1f36c6a>:13: RuntimeWarning: invalid value encountered in true_divide
  vecs /= count
<ipython-input-18-9457d1f36c6a>:13: RuntimeWarning: invalid value encountered in true_divide
  vecs /= count
<ipython-input-18-9457d1f36c6a>:13: RuntimeWarning: invalid value encountered in true_divide
  vecs /= count
<ipython-input-18-9457d1f36c6a>:13: RuntimeWarning: invalid value encountered in true_divide
  vecs /= count
<ipython-input-18-9457d1f36c6a>:13: RuntimeWarning: invalid value encountered in true_divide
  vecs /= count
<ipython-input-18-9457d1f36c6a>:31: RuntimeWarning: invalid value encountered in true_divide
  vecs /= count


In [19]:
lb = LabelBinarizer()
train_y_oh = lb.fit_transform(train_y)
test_y_oh = lb.transform(test_y)

In [20]:
model_ff = tf.keras.Sequential()
model_ff.add(tf.keras.layers.Dense(100, activation = 'sigmoid'))
model_ff.add(tf.keras.layers.Dense(20, activation = 'sigmoid'))
model_ff.add(tf.keras.layers.Dense(16, activation = 'softmax'))
model_ff.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model_ff.fit(inputs_train, train_y_oh, epochs = 25)
model_ff.evaluate(inputs_test, test_y_oh)

Epoch 1/25
1024/1024 [==============================] - 3s 2ms/step - loss: 2.3460 - accuracy: 0.2477
Epoch 2/25
1024/1024 [==============================] - 3s 3ms/step - loss: 2.2666 - accuracy: 0.2920
Epoch 3/25
1024/1024 [==============================] - 2s 2ms/step - loss: 2.2298 - accuracy: 0.2955
Epoch 4/25
1024/1024 [==============================] - 2s 2ms/step - loss: 2.2088 - accuracy: 0.3007
Epoch 5/25
1024/1024 [==============================] - 2s 2ms/step - loss: 2.1945 - accuracy: 0.3018
Epoch 6/25
1024/1024 [==============================] - 2s 2ms/step - loss: 2.1805 - accuracy: 0.3059
Epoch 7/25
1024/1024 [==============================] - 2s 2ms/step - loss: 2.1697 - accuracy: 0.3067
Epoch 8/25
1024/1024 [==============================] - 2s 2ms/step - loss: 2.1606 - accuracy: 0.3094
Epoch 9/25
1024/1024 [==============================] - 3s 3ms/step - loss: 2.1528 - accuracy: 0.3098
Epoch 10/25
1024/1024 [==============================] - 2s 2ms/step - loss: 2.143

[2.0915768146514893, 0.3334554433822632]

In [29]:
inputs_train = np.empty([100,10,1])
for i in range(len(train_x)):
  vecs = np.zeros(100)
  count = 0
  for word in train_x.iloc[i]:
    word = word.lower()
    word = re.sub(r'[^\w\s]', '', word)
    try: 
      if count == 0:
        vecs += glove[word]
      elif count < 10:
        vecs = np.column_stack((vecs, glove[word]))
      else: 
        break
      count += 1
    except KeyError:
      pass
  if count == 0:
    zeroes = np.zeros((100, 9 - count))
    vecs = np.column_stack((vecs, zeroes))
  elif count != 10:
    zeroes = np.zeros((100, 10 - count))
    vecs = np.column_stack((vecs, zeroes))
  inputs_train = np.dstack((inputs_train, vecs))
inputs_train = np.delete(inputs_train, 0, 2)

inputs_test = np.empty([100,10,1])
for i in range(len(test_x)):
  vecs = np.zeros(100)
  count = 0
  for word in test_x.iloc[i]:
    word = word.lower()
    word = re.sub(r'[^\w\s]', '', word)
    try: 
      if count == 0:
        vecs += glove[word]
      elif count < 10:
        vecs = np.column_stack((vecs, glove[word]))
      else: 
        break
      count += 1
    except KeyError:
      pass
  if count == 0:
      zeroes = np.zeros((100, 9 - count))
      vecs = np.column_stack((vecs, zeroes))
  elif count != 10:
      zeroes = np.zeros((100, 10 - count))
      vecs = np.column_stack((vecs, zeroes))
  inputs_test = np.dstack((inputs_test, vecs))
inputs_test = np.delete(inputs_test, 0, 2)

In [32]:
inputs_train = inputs_train.reshape(inputs_train.shape[2], inputs_train.shape[0], inputs_train.shape[1])
inputs_test = inputs_test.reshape(inputs_test.shape[2], inputs_test.shape[0], inputs_test.shape[1])

In [33]:
model_rnn = tf.keras.Sequential()
model_rnn.add(tf.keras.layers.SimpleRNN(10, activation = 'sigmoid'))
model_rnn.add(tf.keras.layers.Dense(20, activation = 'sigmoid'))
model_rnn.add(tf.keras.layers.Flatten())
model_rnn.add(tf.keras.layers.Dense(16, activation = 'softmax'))
model_rnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model_rnn.fit(inputs_train, train_y_oh, epochs = 5)
model_rnn.evaluate(inputs_test, test_y_oh)

Epoch 1/5
1024/1024 [==============================] - 19s 17ms/step - loss: 2.3720 - accuracy: 0.2365
Epoch 2/5
1024/1024 [==============================] - 19s 19ms/step - loss: 2.3310 - accuracy: 0.2438
Epoch 3/5
1024/1024 [==============================] - 20s 20ms/step - loss: 2.3308 - accuracy: 0.2438
Epoch 4/5
1024/1024 [==============================] - 18s 18ms/step - loss: 2.3307 - accuracy: 0.2438
Epoch 5/5
256/256 [==============================] - 2s 7ms/step - loss: 2.3486 - accuracy: 0.2415


[2.3486294746398926, 0.24151404201984406]

In [3]:
row = "The omicron variant is more contagious than delta, but so far it has been quite mild for majority of patients. Some have even speculated it might be a kind of natural vaccine, with which the coronavirus might spread to cover sufficient people, induce herd immunity and finally end Covid-19. Given its mild symptoms (and ‚Äòprecaution fatigue‚Äô), some are either not taking enough care, or even wonder if it‚Äôs not a bad idea to get infected and done with it. It‚Äôs a bad idea. The counter-intuitive logic is: If one is going to get the infection, this may not be a bad time since symptoms are mild, and also one would develop antibodies which would protect against any future waves ‚Äì at least for a while. The ‚Äònatural vaccine‚Äô theory has also been doing rounds in some places, but experts have dismissed the claim: https://timesofindia.indiatimes.com/india/will-omicron-act-as-natural-vaccine-experts-weigh-in/articleshow/88756578.cms and https://www.hindustantimes.com/india-news/omicron-acts-as-natural-vaccine-experts-say-notion-rooted-in-pandemic-fatigue-101641107770257.html The trouble is, omicron is as deadly as any other variant, with severe symptoms, for people with co-morbidities and other vulnerabilities. Plus, also consider: https://www.washingtonpost.com/opinions/2022/01/12/want-just-get-omicron-get-it-over-with-heres-why-you-shouldnt/ https://indianexpress.com/article/explained/omicron-coronavirus-risks-explained-7720793/ * Hospitalizations numbers are rising, and one should rather not add to the immense pressure under which the public healthcare system is trying to work. Resources are under unprecedented strain. * You may be a healthy person without any vulnerability and may end up with only mild symptoms (especially if you are vaccinated), but you will add up continuing the virus chain, passing it on to others, some of whom may be at higher risk. If one person makes all attempts to protect oneself, there‚Äôs a little bit higher chance of breaking the chain. * If omicron continues to gain currency, it can lead to further mutation of the virus and future editions may not be so ‚Äòbenign‚Äô: https://www.cnbc.com/2021/12/07/pfizer-ceo-says-omicron-appears-milder-but-spreads-faster-and-could-lead-to-more-mutations.html  * The term ‚Äòmild‚Äô is misleading. There‚Äôs bad cold, about a week of fever, congestion, sore throat and lingering fatigue. It‚Äôd keep the whole family in isolation. One doesn‚Äôt know its long-term aftereffects. * As for getting protection against the next variant, better to keep faith in the more effective treatment and vaccines coming up soon. Scientists are working in that direction. More information: https://www.nature.com/articles/d41586-021-03614-z https://www.bbc.com/news/health-59769967 https://www.usatoday.com/story/news/2021/12/28/omicron-good-news-may-go-quickly-pose-little-threat-vaccinated/9029369002/ https://www.theguardian.com/science/2021/dec/18/is-there-any-good-news-at-all-on-omicron-yes-there-are-small-signs-of-hope"
re.sub(r'[^\x00-\x7F]',' ', row)

'The omicron variant is more contagious than delta, but so far it has been quite mild for majority of patients. Some have even speculated it might be a kind of natural vaccine, with which the coronavirus might spread to cover sufficient people, induce herd immunity and finally end Covid-19. Given its mild symptoms (and    precaution fatigue   ), some are either not taking enough care, or even wonder if it   s not a bad idea to get infected and done with it. It   s a bad idea. The counter-intuitive logic is: If one is going to get the infection, this may not be a bad time since symptoms are mild, and also one would develop antibodies which would protect against any future waves     at least for a while. The    natural vaccine    theory has also been doing rounds in some places, but experts have dismissed the claim: https://timesofindia.indiatimes.com/india/will-omicron-act-as-natural-vaccine-experts-weigh-in/articleshow/88756578.cms and https://www.hindustantimes.com/india-news/omicron-